# Mentorship: `mentor_lists`
This notebook updates `urbanecm_growth.mentor_lists` with mentor lists configured at all Growth wikis. It runs regularly.

### License statement
Copyright 2021 Martin Urbanec (murbanec-ctr@wikimedia.org)

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [ ]:
from wmfdata import mariadb, utils, spark, hive

import pandas as pd
import requests

In [ ]:
wikis_df = spark.run('SELECT * FROM canonical_data.wikis')
growth_wikis = utils.get_dblist('growthexperiments')
pandaData = {
    'wiki_db': [],
    'domain': [],
    'auto_list': [],
    'manual_list': [],
}

for wiki in growth_wikis:
    domain = None
    tmp = list(wikis_df.loc[wikis_df.database_code == wiki].domain_name)
    if len(tmp) == 0:
        continue
    domain = tmp[0]
    
    try:
        geConfig = requests.get(
            'https://{domain}/w/index.php?title=MediaWiki:GrowthExperimentsConfig.json&action=raw'.format(
                domain=domain
            )
        ).json()
    except:
        print('ERROR fetching wiki config: {wiki}'.format(wiki=wiki))
        continue
    
    pandaData['wiki_db'].append(wiki)
    pandaData['domain'].append(domain)
    pandaData['auto_list'].append(geConfig.get('GEHomepageMentorsList', ''))
    pandaData['manual_list'].append(geConfig.get('GEHomepageManualAssignmentMentorsList', ''))

mentors_df = pd.DataFrame(pandaData)
mentors_df.to_csv('/home/urbanecm/tmp/mentor_lists.tsv', index=False)

In [ ]:
!sed 1d -i /home/urbanecm/tmp/mentor_lists.tsv

In [ ]:
hive.run([
    'TRUNCATE TABLE urbanecm_growth.mentor_lists',
    '''
    LOAD DATA LOCAL INPATH "/home/urbanecm/tmp/mentor_lists.tsv"
    OVERWRITE INTO TABLE urbanecm_growth.mentor_lists
    '''
])